In [25]:
import os
import re
import collections

import numpy as np

In [26]:
#constants
DATA_FOLDER = "data/"

In [27]:
with open(os.path.join(DATA_FOLDER, "WarAndPeace.txt"), "r") as file:
    war_and_peace = file.read()

In [28]:
reg = re.compile(r"[^а-я ]+")

def preprocess_text(text, reg = reg):
    text = text.lower()
    return re.sub(reg, "", text)

In [29]:
ru_data = preprocess_text(war_and_peace)

1) Реализуйте базовый частотный метод по Шерлоку Холмсу:
    - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    - расшифруйте их таким частотным методом.


In [30]:
def get_frequncy_of_n_gram(text, n_gram = 1):
    length = len(text)
    counter = collections.Counter()
    for i in range(len(text)):
        counter[text[i:i + n_gram]] += 1
    normalized_dict = dict({k:v / length for k,v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}) 
    return normalized_dict

In [31]:
ru_characters_frequency = get_frequncy_of_n_gram(text = ru_data)
print(f"Russian characters frequency:\n\n {ru_characters_frequency}")

Russian characters frequency:

 {' ': 0.1704868147910879, 'о': 0.09437233970469568, 'а': 0.0696204286039879, 'е': 0.0654779137740928, 'и': 0.055189385306238105, 'н': 0.05408214807103566, 'т': 0.047152290548906316, 'с': 0.04331622941832316, 'л': 0.042005716362471586, 'в': 0.03822817402874197, 'р': 0.037837022070826225, 'к': 0.02976450804171466, 'д': 0.025235461158918573, 'м': 0.02454709531172039, 'у': 0.02379867069933042, 'п': 0.02132394157976112, 'я': 0.019214184956357298, 'г': 0.017212226116631042, 'ь': 0.016166587614958636, 'ы': 0.015758496005322132, 'з': 0.014786775983885772, 'б': 0.014337105229116489, 'ч': 0.011317227317806346, 'й': 0.009563203380538496, 'ж': 0.008408227126850272, 'ш': 0.007838438841697415, 'х': 0.007083854355954442, 'ю': 0.005382189342187125, 'ц': 0.003355591009048854, 'э': 0.002508608423010823, 'щ': 0.002331512064111962, 'ф': 0.0018618217209454173, 'ъ': 0.00043581103972502327}


In [32]:
test = 'Ах, не говорите мне про Австрию! ' + \
                        'Я ничего не понимаю, может быть, но Австрия никогда не хотела и не хочет войны. Она предает нас. Россия одна должна быть спасительницей Европы. ' + \
                        'Наш благодетель знает свое высокое призвание и будет верен ему. Вот одно, во что я верю. Нашему доброму и чудному государю предстоит величайшая роль в мире, ' + \
                        'и он так добродетелен и хорош, что Бог не оставит его, и он исполнит свое призвание задавить гидру революции, которая теперь еще ужаснее в лице этого убийцы и злодея. '

In [33]:
test = preprocess_text(test)
print(test)

ах не говорите мне про австрию я ничего не понимаю может быть но австрия никогда не хотела и не хочет войны она предает нас россия одна должна быть спасительницей европы наш благодетель знает свое высокое призвание и будет верен ему вот одно во что я верю нашему доброму и чудному государю предстоит величайшая роль в мире и он так добродетелен и хорош что бог не оставит его и он исполнит свое призвание задавить гидру революции которая теперь еще ужаснее в лице этого убийцы и злодея 


In [34]:
def get_encoding_dict(frequencies):
    keys = list(frequencies.keys())
    shuffled_keys = keys.copy()
    np.random.shuffle(shuffled_keys)
    encoding_dict = dict(zip(keys, shuffled_keys))
    return encoding_dict

    
def get_decoding_dict(encoded_frequncies, frequencies):   
    decoding_dict = dict(list(zip(encoded_frequncies.keys(), frequencies.keys())))
    return decoding_dict


def calculate_accuracy(text1, text2):
    counter = 0
    for t1, t2 in zip(text1, text2):
        if t1 == t2:
            counter += 1
    return counter / len(text1)

In [35]:
def encode(text_to_encode, encoding_dict, n_gram = 1):
    splitted_to_grams = [text_to_encode[i:i + n_gram] for i in range(0, len(text_to_encode), n_gram)]
    result = []
    for gram in splitted_to_grams:
        if(len(gram) < n_gram):
            continue
        result.append(encoding_dict[gram])
    return "".join(result)


def decode(encoded_text, decoding_dict, n_gram = 1):
    splitted_to_grams = [encoded_text[i:i + n_gram] for i in range(0, len(encoded_text), n_gram)]
    result = []
    for gram in splitted_to_grams:
        result.append(decoding_dict[gram])
    return "".join(result)

In [36]:
frequencies = get_frequncy_of_n_gram(ru_data)
dict_for_encoding = get_encoding_dict(frequencies)
encoded_text = encode(test, dict_for_encoding)

frequencies_encoded = get_frequncy_of_n_gram(encoded_text)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
decoded_text = decode(encoded_text, dict_for_decoding)
print(f"Accuracy = {calculate_accuracy(test, decoded_text)}")

Accuracy = 0.345679012345679


2) Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
    - подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
    - проведите тестирование аналогично п.1, но при помощи биграмм.


In [61]:
frequencies = get_frequncy_of_n_gram(ru_data, 2)
dict_for_encoding = get_encoding_dict(frequencies)
encoded_text = encode(test, dict_for_encoding, 2)

frequencies_encoded = get_frequncy_of_n_gram(encoded_text, 2)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
decoded_text = decode(encoded_text, dict_for_decoding, 2)
print(f"Accuracy = {calculate_accuracy(test, decoded_text)}")

Accuracy = 0.10082304526748971


Как видно, с биграммами стало даже хуже

3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
    - предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
    - реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Предлагаю использовать алгоритм Метрополиса-Гастингса. Будем переставлять случайные значения в словаре для декодирования и оценивать правдоподобие как произведение всех вероятностей биграм текста. Если правдоподобие выше для такой перестановки - сохраняем ее, если ниже, то сохраняем с вероятностью отношения нового и старого правдоподобий.   

In [38]:
def func_log_likelihood(encoded_text, dict_for_decoding, frequncies_n_gram, n_gram):
    decoded_text = decode(encoded_text, dict_for_decoding)
    log_likelihood = 0
    for i, _ in enumerate(decoded_text[:-1]):
        bigram = decoded_text[i: i + n_gram]
        probability = frequncies_n_gram.get(bigram, 1 / len(frequncies_n_gram))
        log_likelihood += np.log(probability)
    return log_likelihood

def metropolis_hastings_log_accept(current_likelihood, new_likelyhood):
    if new_likelyhood > current_likelihood:
        return True
    else:
        return np.random.rand() < np.exp(new_likelyhood - current_likelihood)  

def random_permutation(dict_for_decoding):
    new_values = list(dict_for_decoding.values())
    key_1, key_2 = np.random.choice(range(len(list(dict_for_decoding.values()))), 2, replace=False)
    new_values[key_1], new_values[key_2] = new_values[key_2], new_values[key_1]
    new_dict_for_decoding = {key: val for key, val in zip(list(dict_for_decoding.keys()), new_values)}
    return new_dict_for_decoding

def metropolis_hastings(encoded_text, frequencies_bigram, dict_for_decoding, iterations, n_gram):
    current_dict_for_decoding = dict_for_decoding
    best_dict_for_decoding = dict_for_decoding
    best_likelihood = -np.inf
    current_likelihood = func_log_likelihood(encoded_text, dict_for_decoding, frequencies_bigram, n_gram)
  
    for i in range(iterations):
        new_dict_for_decoding = random_permutation(current_dict_for_decoding)
        new_likelyhood = func_log_likelihood(encoded_text, new_dict_for_decoding, frequencies_bigram, n_gram)
        if metropolis_hastings_log_accept(current_likelihood, new_likelyhood):
            current_dict_for_decoding = new_dict_for_decoding
            current_likelihood = new_likelyhood            
        if current_likelihood > best_likelihood:
            best_likelihood = current_likelihood
            best_dict_for_decoding = current_dict_for_decoding
    return best_likelihood, best_dict_for_decoding

In [39]:
frequencies = get_frequncy_of_n_gram(ru_data)
dict_for_encoding = get_encoding_dict(frequencies)
encoded_text = encode(test, dict_for_encoding)

frequencies_encoded = get_frequncy_of_n_gram(encoded_text)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
frequncies_bigram = get_frequncy_of_n_gram(ru_data, 2)

likelihood, dict_for_decoding = metropolis_hastings(encoded_text, frequncies_bigram, dict_for_decoding, 30000, n_gram=2)

In [40]:
decoded_text = decode(encoded_text, dict_for_decoding)

In [41]:
print(likelihood)
print(decoded_text)
print(f"Accuracy = {calculate_accuracy(test, decoded_text)}")

-2655.7698035079948
ак не говорите мне про австрий я ничего не понимай может быть но австрия нихогда не котела и не кочет вошны она предает нас россия одна должна быть спасительницеш европы наю благодетель знает свое высохое призвание и будет верен ему вот одно во что я верй наюему доброму и чудному государй предстоит величашюая роль в мире и он тах добродетелен и корою что бог не оставит его и он исполнит свое призвание задавить гидру револйции хоторая теперь еще ужаснее в лице этого убишцы и злодея 
Accuracy = 0.9567901234567902


4) Расшифруйте сообщение:

In [42]:
message = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [43]:
frequencies = get_frequncy_of_n_gram(ru_data)
frequencies_encoded = get_frequncy_of_n_gram(message)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
frequncies_bigram = get_frequncy_of_n_gram(ru_data, 2)

likelihood, dict_for_decoding = metropolis_hastings(message, frequncies_bigram, dict_for_decoding, 30000, n_gram = 2)

In [44]:
decoded_text = decode(message, dict_for_decoding)
print(f"Likelihood = {likelihood}")
print(decoded_text)

Likelihood = -1253.8571388105913
если вы вимите норзальный или подти норзальный текст у чтого соожшения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный жалл ба послемнее детвертое бамание курса хотя конедно я нидего не ожешаю


Несмотря на мелкие ошибки, текст легко читается

5) Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.


In [45]:
frequencies = get_frequncy_of_n_gram(ru_data)
dict_for_encoding = get_encoding_dict(frequencies)
encoded_text = encode(test, dict_for_encoding)

frequencies_encoded = get_frequncy_of_n_gram(encoded_text)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
frequncies_trigram = get_frequncy_of_n_gram(ru_data, 3)

likelihood, dict_for_decoding = metropolis_hastings(encoded_text, frequncies_trigram, dict_for_decoding, 30000, n_gram = 3)

In [46]:
decoded_text = decode(encoded_text, dict_for_decoding)
print(f"Likelihood = {likelihood}")
print(decoded_text)
print(calculate_accuracy(test, decoded_text))

Likelihood = -3646.6043511448934
ах не говорите мне про австрий я ничего не понимай может быть но австрия никогда не хотела и не хочет вошны она предает нас россия одна должна быть спасительницеш европы наю благодетель знает свое высокое призвание и будет верен ему вот одно во что я верй наюему доброму и чудному государй предстоит величашюая роль в мире и он так добродетелен и хорою что бог не оставит его и он исполнит свое призвание задавить гидру револйции которая теперь еще ужаснее в лице этого убишцы и злодея 
0.9732510288065843


In [53]:
frequencies = get_frequncy_of_n_gram(ru_data)
dict_for_encoding = get_encoding_dict(frequencies)
encoded_text = encode(test, dict_for_encoding)

frequencies_encoded = get_frequncy_of_n_gram(encoded_text)
dict_for_decoding = get_decoding_dict(frequencies_encoded, frequencies)
frequncies_4gram = get_frequncy_of_n_gram(ru_data, 4)

likelihood, dict_for_decoding = metropolis_hastings(encoded_text, frequncies_4gram, dict_for_decoding, 30000, n_gram = 4)

In [54]:
decoded_text = decode(encoded_text, dict_for_decoding)
print(f"Likelihood = {likelihood}")
print(decoded_text)
print(calculate_accuracy(test, decoded_text))

Likelihood = -4448.792710139201
ах не говорите мне про австрий я ничего не понимай может быть но австрия никогда не хотела и не хочет воцны она предает нас россия одна должна быть спасительниюец европы наш благодетель знает свое высокое призвание и будет верен ему вот одно во что я верй нашему доброму и чудному государй предстоит величацшая роль в мире и он так добродетелен и хорош что бог не оставит его и он исполнит свое призвание задавить гидру револйюии которая теперь еще ужаснее в лиюе этого убицюы и злодея 
0.9732510288065843


Кажется, что с ростом длины n-граммы повышается вероятность не найти подходящую перестановку и в итоге не найти хороший максимум правдоподобия, хотя, конечная точность становится выше

6) Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Кроме как пытаться востановить кодировку мне ничего в голову не приходит :)